In [95]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from surprise import *
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import KNNBaseline
from surprise import accuracy
from surprise import BaselineOnly
from surprise.model_selection import train_test_split
from helpers import *
from surprise.model_selection import GridSearchCV
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data loading

In [74]:
from helpers import *

DATA_PATH = 'data/'
dataset_base, dataframe = load_data_surprise(DATA_PATH + 'data_train.csv',0)

In [75]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(dataset_base, test_size=.2)
sample_dataset, sample_dataframe = load_data_surprise(DATA_PATH + 'sampleSubmission.csv',0)
predset = sample_dataset.build_full_trainset().build_testset()
realval_testset = [p[2] for p in testset]

In [76]:
GM_test,GM_pred =global_mean_pred(trainset,testset,predset)

In [77]:
UserM_test,UserM_pred =user_mean_pred(trainset,testset,predset)
ItemM_test,ItemM_pred =item_mean_pred(trainset,testset,predset)

In [78]:
Base_test,Base_pred=train_model(BaselineOnly(),trainset,testset,predset)

Fit model...
Estimating biases using als...
Make predictions...


In [84]:
SVD_test,SVD_pred=train_model(SVD(n_factors=20),trainset,testset,predset)

Fit model...
Make predictions...


In [91]:
SVDpp_test,SVDpp_pred=train_model(SVDpp(),trainset,testset,predset)

Fit model...
Make predictions...


In [109]:
SlopeOne_test,SlopeOne_pred=train_model(SlopeOne(),trainset,testset,predset)

Fit model...
Make predictions...


In [88]:
NMF_test,NMF_pred=train_model(NMF(biased=True,n_factors=5),trainset,testset,predset)

Fit model...
Make predictions...


In [118]:
KNN_test,KNN_pred=train_model(KNNBaseline(k=10),trainset,testset,predset)

Fit model...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Make predictions...


KeyboardInterrupt: 

In [110]:
pred=np.concatenate((GM_test,UserM_test,ItemM_test,Base_test,SVD_test,SVDpp_test,SlopeOne_test,NMF_test),axis=1)

In [116]:
x=coeff(pred,realval_testset)
print(x)

[ 0.04227961 -0.09291693  0.04421564  0.23027703  0.19990574  0.33498601
  0.61322071 -0.37169863]


In [117]:
calculate_poud_rmse_round(x,realval_testset,pred.T)

1.0335465209990513

In [ ]:
[ 0.01463913 -0.06141506  0.01556897  0.71519193  0.26279121  0.05578622]

In [67]:
print(np.shape(NMF_test))
print(np.shape(GM_test))

(1176952, 1)
(235391, 1)


## 4.2 Final Model

In [16]:
fullset_base = dataset_base.build_full_trainset()

In [48]:
fullset_base = dataset_base.build_full_trainset()
algo_svd= SVD()
algo_svd.n_factors = 20
algo_svd.fit(fullset_base)
predic_svd = algo_svd.test(sub_data_set.build_testset())

In [49]:
algo_svdpp = SVDpp()
algo_svdpp.fit(fullset_base)
predic_svdpp = algo_svdpp.test(sub_data_set.build_testset())

In [51]:
algo_KNN = KNNBaseline()
algo_KNN.fit(fullset_base)
predic_KNN = algo_KNN.test(sub_data_set.build_testset())

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [52]:
bsl_options = {'method': 'als',
               'n_epochs': 200,
               'reg_u': 10,
               'reg_i': 15
               }
algo_base = BaselineOnly(bsl_options=bsl_options)
algo_base.fit(fullset_base)
predic_base = algo_base.test(sub_data_set.build_testset())

Estimating biases using als...


In [53]:
algo_NMF = NMF()
algo_NMF.biased=True
algo_NMF.n_factors=2
algo_NMF.fit(fullset_base)
predic3_NMF= algo_NMF.test(sub_data_set.build_testset())

In [54]:
usermean=np.array([dict_mean[p[0]] for p in predic_base])
usermean=np.reshape(usermean,(len(usermean),1))
itemmean=np.array([dict_mean2[p[1]] for p in predic_base])
itemmean=np.reshape(itemmean,(len(itemmean),1))

In [63]:
usermean[0]

array([3.75773196])

In [50]:
len(predic_NMF)

1176952

In [51]:
fullset_base2 = dataset_base.build_full_trainset().build_testset()
pm2=[]
x=[ 0.42584923,  0.75550288, -0.89979921, -0.24605356,  0.86926427, 0.92551532, -0.82951495]
for i in range(len(predic_svd)):
    r=round(x[0]*predic_svd[i][3]+x[1]*pred_Slope[i][3]+x[2]*predic_base[i][3]+x[3]*predic_NMF[i][3]+x[4]*usermean_test[i][0]+x[5]*itemmean_test[i][0]+x[6]*trainset.global_mean)
    if r>5:
        pm2.append(5)
    elif r<1:
        pm2.append(1)
    else:
        pm2.append(r)


## 4.3 Write output submission to CSV

In [52]:
import csv
with open('submission.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Id', 'Prediction'])
    for i in range(len(predic_base)):
        writer.writerow([predic_base[i][0] + '_' + predic_base[i][1], pm2[i]])

# 5.Analyse Submission

In [80]:
sub_dataset, sub_dataframe = load_data_surprise('submission.csv',0)


In [81]:
g   = sub_dataframe.groupby('Prediction')
num_sub = g.apply(lambda x: len(x)/len(sub_dataframe))
print(num_sub)

Prediction
1.0    0.000238
2.0    0.012426
3.0    0.226358
4.0    0.656191
5.0    0.104788
dtype: float64
